pip install -U Flask# Project 2 

In [ ]:
pip install -U Flask

### First: Import libraries 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split #هذي المكتبه تاخذ الداتا اللي للتدريب 
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold 

In [2]:
## CSV file 

datafile = "/Users/sarah/Desktop/exp/0.csv"
df=pd.read_csv(datafile)

In [ ]:
## Examine the columns, look at missing data
# df.info() , df.head() , df.shape , df.columns

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df[['Neighborhood', 'space']]

### Third: pre - processing

In [ ]:
# duplcate + outliers + null // DONE !!

### Fourth: linear regrestion 

In [3]:
from sklearn.preprocessing import OneHotEncoder
X=df[['Neighborhood', 'space']]
y=df.price
X=pd.get_dummies(X)

In [4]:
# هنا خليت الترين ٨٠ بالميه والتيست ٢٠ بالميه
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size= .20 ,random_state=4)

In [ ]:
print(y_test)

In [5]:
m = LinearRegression()

In [6]:
# train the model 80 %
m.fit(X_train, y_train)

LinearRegression()

In [ ]:
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))

In [ ]:
plt.scatter(X_train, y_train)
plt.plot(X_train, m.predict(X_train), color ='y')
plt.show()

In [ ]:
plt.scatter(X_test, y_test)
plt.plot(X_test, m.predict(X_test), color ='r')
plt.show()

In [ ]:
# Here we will break the data into 3 portions: 
# 60% for training,
# 20% for validation (used to select the model), 
# 20% for final testing evaluation.
# X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [7]:
# Here we will break the data into 3 portions:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
X_train.columns

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
type(X_train)

In [ ]:
#df.describe()

In [ ]:
plt.hist(X_train)

In [ ]:
#set up the 3 models we're choosing from:

lm = LinearRegression()

#Feature scaling for train, val, and test so that we can run our ridge model on each

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

lm_reg = Ridge(alpha=1) # هذي ليه ؟؟ 

#Feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()


In [ ]:
# Now we can train, validate, and test.

In [ ]:
#validate

lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

In [ ]:
# Final step we retrain it on the entire chunk of train/val data and see how it does on test data

In [ ]:
lm.fit(X,y)
print(f'Linear Regression test R^2: {lm.score(X_test, y_test):.3f}')

In [ ]:
plt.scatter(X_test, y_test)

In [ ]:
#Not terrible!

# This is a pretty solid selection method, but we can make it even more rigorous using cross-validation.

In [ ]:
# Here we will break the data into 2 portions: 
#80% for a cross-validated training process, 
#and 20% for final testing evaluation.

In [ ]:
# k-fold CV

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

In [ ]:
#run the CV

kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
# Since k-fold is more reliable than a single validation set, we select the ridge regression model. 
#This shows the dangers of relying on simple validation methods, especially when our sample sizes are small.


## This step fits the Standard Scaler to the training data
## Essentially it finds the mean and standard deviation of each variable in the training set

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

#  K-fold, in a Less Manual Way with Scikit-learn : 

In [ ]:
from sklearn.model_selection import cross_val_score
lm = LinearRegression()

cross_val_score(lm, X, y, # estimator, features, target
                cv=5, # number of folds 
                scoring='r2') # scoring metric

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state = 1000)

print(np.mean(cross_val_score(lm, X, y, cv=kf, scoring='r2')))
print(np.mean(cross_val_score(lm_reg, X, y, cv=kf, scoring='r2')))

In [ ]:
pip install -U Flask

In [17]:
columns=X.columns.values

In [42]:
new_x= np.zeros(188)
new_x[0]= 2300
new_x[np.where(columns=='Neighborhood_المرسلات')[0][0]]=1
new_x

array([2.3e+03, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       1.0e+00, 0.0e

In [43]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route("/")
def predict():
    new_x= np.zeros(188)
    new_x[0]= int(request.args.get('meter'))
    neigh=request.args.get('neigh')
    new_x[np.where(columns==neigh)[0][0]]=1
 

    return {'price':m.predict([new_x])[0]}

In [ ]:
app.run(host='0.0.0.0',port=5004)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.100.45:5004/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2021 01:13:21] "GET /?meter=400&neigh=Neighborhood_عرقه HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 07:59:05] "GET /?meter=400&neigh=Neighborhood_العقيق HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2021 08:30:47] "GET /?meter=400&neigh=Neighborhood_العقيق HTTP/1.1" 200 -
